# Level Of Community Transmission

### https://covid.cdc.gov/covid-data-tracker/#county-view

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
import requests
import io

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
s = requests.Session()
response = s.get("https://covid.cdc.gov/covid-data-tracker/content/CoronaViewJson_01/fips-codes-covid-tracker.csv")

In [ ]:
fips = pd.read_csv(io.StringIO(response.text))
subdivisions = pycountry.subdivisions.get(country_code='US')
subdivision_dict = {subdivision.name: subdivision.code for subdivision in subdivisions}

In [ ]:
response = s.get("https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_latest_external_data")
transition_df = pd.DataFrame(json.loads(response.text)["integrated_county_latest_external_data"])
transition_df["fips_code"] = transition_df["fips_code"].astype(str)
transition_df["fips_code"] = transition_df["fips_code"].str.zfill(5)

In [ ]:
df = pd.DataFrame()

for i in range(len(fips)):
    area = fips.iloc[i]
    fips_code = str(area["fips_code"]).zfill(5)
    
    # get response
    url = "https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_fips_{fips_code}_external".format(fips_code=fips_code)
    response = s.get(url)
    
    # create df for area
    try:
        tmp_df = pd.read_json(io.StringIO(json.dumps(json.loads(response.text)["integrated_county_timeseries_external_data"])))
    except:
        continue
    tmp_df["state_name"] = area["state_name"]
    tmp_df["county"] = area["county"]
    tmp_df["fips_code"] = fips_code
    tmp_df["svi"] = transition_df.loc[transition_df["fips_code"] == fips_code]["svi_rank"]
    tmp_df["ISO3166_2"] = subdivision_dict[area["state_name"]][-2:]
    tmp_df = tmp_df.drop(columns=["state"])
    tmp_df["new_cases_7_day_rolling_average"] = tmp_df.replace({"suppressed": np.nan})
    tmp_df["new_deaths_7_day_rolling_average"] = tmp_df.replace({"suppressed": np.nan})
    df = df.append(tmp_df, ignore_index=True)

In [ ]:
df["Last_Updated_Date"] = datetime.utcnow()
df['Last_Reported_Flag'] = df["date"].max() == df["date"]

In [ ]:
df = df.sort_values(by=['state_name', 'date'])

In [ ]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [ ]:
df = df.rename(columns={
    "fips_code": "FIPS",
    "state_name": "PROVINCE_STATE",
    "county": "COUNTY"
})

In [ ]:
df.dtypes

In [ ]:
df["new_cases_7_day_rolling_average"].unique()

In [ ]:
df.to_csv(output_folder + "COMMUNITY_TRANSMISSION.csv", index=False, columns=[
    "FIPS",
    "PROVINCE_STATE",
    "COUNTY",
    "ISO3166_2",
    "new_cases_7_day_rolling_average",
    "new_deaths_7_day_rolling_average",
    "new_test_results_reported_7_day_rolling_average",
    "admissions_covid_confirmed_7_day_rolling_average",
    "percent_adult_inpatient_beds_used_confirmed_covid",
    "percent_adult_icu_beds_used_confirmed_covid",
    "report_date_window_start",
    "report_date_window_end",
    "community_transmission_level",
    "svi",
    "date",
    "Last_Updated_Date",
    "Last_Reported_Flag"
])